### Thinning 

- Two functions one for neighbouring values and the other for transition.
- The neighbours function list value of the central pixel of 3$\times$3 neighborhood, it uses image, central pixel index, and returns the surrounding values of the central pixel.
- Transition uses the list of the neighbouring values and sums up the transition from 0 to 1, 
- The Zhang-Suen Thinning Algorithm uses the results of Transition function to calculate the conditions for deletion operation. 

In [ ]:
def neighbours(x,y,image):
    "Return 8-neighbours of image point P1(x,y), in a clockwise order"
    img = image
    x_1, y_1, x1, y1 = x-1, y-1, x+1, y+1
    return [img[x_1][y], img[x_1][y1], img[x][y1], img[x1][y1],     # P2,P3,P4,P5
                img[x1][y], img[x1][y_1], img[x][y_1], img[x_1][y_1]]    # P6,P7,P8,P9
#print()
def transitions(neighbours):
    "No. of 0,1 patterns (transitions from 0 to 1) in the ordered sequence"
    n = neighbours + neighbours[0:1]      # P2, P3, ... , P8, P9, P2
    return sum( (n1, n2) == (0, 1) for n1, n2 in zip(n, n[1:]) )  # (P2,P3), (P3,P4), ... , (P8,P9), (P9,P2)


In [ ]:
def Thinning(image,IMAX):
    "the Zhang-Suen Thinning Algorithm"
    Image_Thinned = image.copy()      # deepcopy to protect the original image
    changing1 = changing2 = 1
    count = 0#  the points to be removed (set as 0)
    #while changing1 or changing2: 
    while (count < IMAX):#  iterates until no further changes occur in the image
        # Step 1
        changing1 = []
        rows, columns = Image_Thinned.shape               # x for rows, y for columns
        for x in range(1, rows - 1):                     # No. of  rows
            for y in range(1, columns - 1):            # No. of columns
                P2, P3, P4, P5, P6, P7, P8, P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1     and    # Condition 0: Point P1 in the object regions 
                    2 <= sum(n) <= 6   and    # Condition 1: 2<= N(P1) <= 6
                    transitions(n) == 1 and    # Condition 2: S(P1)=1  
                    P2 * P4 * P6 == 0  and    # Condition 3   
                    P4 * P6 * P8 == 0):         # Condition 4
                    changing1.append((x,y))
        for x, y in changing1: 
            Image_Thinned[x][y] = 0
        # Step 2
        changing2 = []
        for x in range(1, rows - 1):
            for y in range(1, columns - 1):
                P2, P3, P4, P5, P6, P7, P8, P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1   and      # Condition 0
                    2 <= sum(n) <= 6  and       # Condition 1
                    transitions(n) == 1 and     # Condition 2
                    P2 * P4 * P8 == 0 and       # Condition 3
                    P2 * P6 * P8 == 0):         # Condition 4
                    changing2.append((x,y))    
        for x, y in changing2: 
            Image_Thinned[x][y] = 0
        count = count + 1
    return Image_Thinned



### Hough Circle transform
- Hough Transfrom for circles uses the representation of circles center coordinates and radius to identify circle. 
- For any point on a circle and given center of coordinates the radius of the circle can be calculated
- For a given pixel and all possible centers around the pixels the corresponding radius can be calculated and stored in Accumulator
- Pixels with the same radius in the accumulator would mean they lie on the same circle, hence a radius with maximum pixel counts in the accumulator can be identified as circle on th image
- In the function below the accumulator is 3D matrix to account the coordinates of the center and radius 


In [22]:
def largest_indices(ARRAY, n):
    """
    Returns the n index combinations referring to the 
    largest values in a numpy array.
    """
    flat = ARRAY.flatten()   ## covert ARRAY into 1D array (Flat),lists all elements of ARRAY in one row
    indices = np.argpartition(flat, -n)[-n:] ## Partition the 1D and retruns the index of the partitioned vector which is not sorted. To make the top n highest values of the flat -n is used otherwise it will partition based on the least n values. So we must count from the end to get the highest point. [-n:] is to list only the top n indexes 
    indices = indices[np.argsort(-flat[indices])] ## np.argsort will sort the flat index but from higest to the lowest since there is - and returns the index of the sorted vector, using this index values "indices" is sorted 
    return np.unravel_index(indices, ARRAY.shape) ## It will convert the indices of a row array into tuples(coordinates) of the 2D array.
def hough_circles_loops(imgBIN, Nx, Ny, Nr, K = 5, rmin = 1, rmax = 100):
    '''
    Computes the Hough transform to detect circles in the binary 
    image I (of size M × N ), using Nx, Ny, Nr discrete steps for the center coordinates 
    and radius, respectively. Returns the list of parameter triplets (xi, yi, ri) 
    for the K strongest circles found.
    --- direct implementation of our pseudo code, with if / loops 
        -> VEEEEERY SLOW! ---
    '''
    # Initalise the Accumulator Array: 
    Acc = np.zeros((Nx, Ny, Nr))
    r_increment = (rmax-rmin)/Nr
    r = np.arange(rmin, rmax, r_increment)
    xbar = range(Nx)
    ybar = range(Ny)
    # collect the non-zero / foreground elements: 
    nzi = np.nonzero(imgBIN) #returns the index of the nonzero elements in the imgBIN
    y = nzi[0][:]
    x = nzi[1][:]
    print("# relevant foreground pixels: %i" %(x.shape[0]) )
    
    # Filling the Accumulator Array:       
    for cntp in range(x.shape[0]):
          print("Testing foreground pixel %i of %i" %(cntp, x.shape[0]))    
          for cntx, xi in enumerate(xbar):
              for cnty, yi in enumerate(ybar):
                  for cntr, ri in enumerate(r):
                      if np.square(x[cntp] - xi) + np.square(y[cntp] - yi) - np.square(ri) < 1e-16:
                          Acc[cntx, cnty, cntr]  = Acc[cntx, cnty, cntr] + 1
                          
    MaxIDX = largest_indices(Acc, K)
    MaxX = MaxIDX[0][:] 
    MaxY = MaxIDX[1][:] 
    MaxR = r_increment * MaxIDX[2][:] + rmin
    
    return Acc, MaxIDX, MaxX, MaxY, MaxR 



In [33]:

def hough_circles(imgBIN, Nx, Ny, Nr, K = 5, rmin = 1, rmax = 100):
    '''
    Computes the Hough transform to detect circles in the binary 
    image I (of size M × N ), using Nx, Ny, Nr discrete steps for the center coordinates 
    and radius, respectively. Returns the list of parameter triplets (xi, yi, ri) 
    for the K strongest circles found.
    --- exchanged the loops by a 3D Meshgrid. about 1500 times faster, but still
        slower than OpenCV implementations... ---
    '''
    # Initalise the Accumulator Array: 
    Acc = np.zeros((Ny, Nx, Nr))
    r_increment = (rmax-rmin)/Nr
    r = np.arange(rmin, rmax, r_increment)
    # print(r)
    xbar = range(Nx)
    ybar = range(Ny)
    # collect the non-zero / foreground elements: 
    nzi = np.nonzero(imgBIN)
    y = nzi[0][:]
    x = nzi[1][:]
    print("Relevant foreground pixels: %i" %(x.shape[0]) )
    
    # Initialise the coordinate meshgrid
    XBAR, YBAR, RBAR = np.meshgrid(xbar, ybar, r)
    
    # Filling the Accumulator Array:      
    for cntp in range(x.shape[0]):
        # Perform the logical operation on the whole Meshgrid at once: 
        lala = (np.square(x[cntp] - XBAR) + np.square(y[cntp] - YBAR)) - np.square(RBAR) < 1e-16
        Acc = Acc + lala        
                  
    MaxIDX = largest_indices(Acc, K)
    MaxX = MaxIDX[0][:] 
    MaxY = MaxIDX[1][:] 
    MaxR = r_increment * MaxIDX[2][:] + rmin
    
    return Acc, MaxIDX, MaxX, MaxY, MaxR

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import OIP21_lib_ImageProcessing_V6 as ip
import matplotlib.image as mpimg

pics = ['Circle.png',          #0
        'SingleTCell.png',      #1
        'Coins2.png',           #2           
        '1Coin.png',            #3
        '2Coins.png']           #4
                 
PicNUM = 3 # If you want to load an image, pick one of the above...
  
NoLoops = True
 ## For Pics[0] & Pics[1]
# img, imgRED, imgGREEN, imgBLUE = ip.load_image(pics[PicNUM])
# imgINT = ip.convert2LUM(imgRED, imgGREEN, imgBLUE)

## For Pics[2:]

img = mpimg.imread(pics[PicNUM])
imgINT = (img*255).astype(np.uint8)#ip.convert2LUM(imgRED, imgGREEN, imgBLUE)

N, M = imgINT.shape

Nx = N#(np.floor(M/4)).astype(np.int)
Ny = M#(np.floor(N/4)).astype(np.int)
K = 20
rmin = 1
rmax = 35
Nr = (rmax-rmin)+1

imgBIN = ip.threshold_binary(imgINT,80)
ip.plot_image(imgBIN, title='Original Binary Image (Lumosity)', vmax = 1, vmin = 0)
imgBIN, Phi, IDX, IDY = ip.detect_edges(imgBIN, Filter='ISobel')
imgBIN = ip.threshold_binary(imgBIN*255,120)
ip.plot_image(imgBIN, title='Original Binary Image (Lumosity)', vmax = 1, vmin = 0)
imgBIN = ip.Thinning(imgBIN)
ip.plot_image(imgBIN, title='Original Binary Image (Lumosity)', vmax = 1, vmin = 0)

if NoLoops:
    Acc, MaxIDX, MaxX, MaxY, MaxR  = hough_circles(imgBIN, Nx, Ny, Nr, K, rmin, rmax)
else:
    Acc, MaxIDX, MaxX, MaxY, MaxR = hough_circles_loops(imgBIN, Nx, Ny, Nr, K, rmin, rmax)
print(MaxIDX)
print(MaxX)
print(MaxY)
print(MaxR)    


# # ---------------------------------
# # Display the RESULTS: 

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,20))
ax1.imshow(imgINT, cmap=plt.cm.gray, extent = [0,M,0,N])
#plt.gca().invert_yaxis()
for cnt in range(K):
    circle = plt.Circle((MaxY[cnt], N-MaxX[cnt]), MaxR[cnt], color='g', fill=False)
    ax1.add_artist(circle)
ax1.set_title('Original image + detected circles.')
ax2.imshow(np.transpose(Acc[:,:,30]), cmap=plt.cm.gray, origin='lower')
ax2.set_title('Accumulator Array')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '1Coin.png'